In [59]:
import pandas as pd
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import numpy as np

path_weather = r'C:\Users\20177480\Desktop\Case Studies\Heijmans Infra (Modeling and Simulation)\Data\Weather.xlsx'
df=pd.read_excel(path_weather)
df['timestamp'] = pd.to_datetime(df['timestamp'], format="%d-%m-%Y %H:%M.%S")
df = df[df['timestamp']>'23-10-2017 09:00:00']
df = df.set_index('timestamp')
df_daily_mean = df.resample('D').mean()
del df['humidity']
df['datetime1'] = df.index

## Detrend the signal (x-x' where x' is the daily mean)

In [60]:
def detrend(r):
    temp = r['temperature']
    index = r['datetime1']
    date_1 = index.date()
    baseline = df_daily_mean[df_daily_mean['datevalue']==date_1]['temperature'].values[0]
    corrected_temp = temp - baseline
    return corrected_temp

df_daily_mean['datevalue'] = df_daily_mean.index
df['temperature_detrend'] = df.apply(detrend,axis=1)

## Get Features of ambient temperature

In [61]:
df_temperature = df[['temperature']]
df_features['average'] = df_temperature.resample('D').mean()
df_features['maximum'] = df_temperature.resample('D').max()
df_features['minimum'] = df_temperature.resample('D').min()

## FFT of x-x' 

In [74]:
from scipy.fftpack import fft, ifft
from scipy import fft, arange, signal
from plotly import tools

#temperature_fft = df['temperature_detrend'].values
temperature_fft = df_features['average'].values
fs = 1 # samples per day
t = arange(len(temperature_fft))/float(fs) + 1
f, Pxx = signal.periodogram(temperature_fft, fs = fs)
fft_signal= fft(temperature_fft)

## Plot

In [79]:
trace01 = go.Scatter(
    x=df['datetime1'],
    y=df['temperature']
)
trace02 = go.Scatter(
    x=df_features.index,
    y=df_features['maximum']
)
trace03 = go.Scatter(
    x=df_features.index,
    y=df_features['average']
)
trace04 = go.Scatter(
    x=df_features.index,
    y=df_features['minimum']
)
trace2 = go.Scatter(
    x=f,
    y=Pxx,
    xaxis='x3',
    yaxis='y3'
)
fig = tools.make_subplots(rows=2, cols=1, subplot_titles=('Temperature - Daily Average','FFT - Detrend Signal'))
fig.append_trace(trace01, 1, 1)
fig.append_trace(trace02, 1, 1)
fig.append_trace(trace03, 1, 1)
fig.append_trace(trace04, 1, 1)
fig.append_trace(trace2, 2, 1)
fig['layout'].update(height=700, width=1600)
plot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]



'file://C:\\Users\\20177480\\temp-plot.html'